In [214]:
from searchtweets import collect_results, gen_rule_payload, load_credentials, ResultStream
from text_cleaning import clean_text
import pandas as pd
import numpy as np
premium_search_args = load_credentials(filename="./search_tweets_creds.yaml",yaml_key="search_tweets_api",env_overwrite=False)

In [228]:
username = "MaximeBernier"
file_path = "../../data/{}_data.csv".format(username)


In [229]:
rule = gen_rule_payload("from:{} -is:retweet lang:en".format(username),
                        from_date="2018-10-21", #UTC 2018-10-21 00:00
                        to_date="2019-10-21",#UTC 2017-10-30 00:00
                        results_per_call=500)
rs = ResultStream(rule_payload=rule,
                  max_results=10000,
                  max_pages=500,
                  **premium_search_args)
tweets = list(rs.stream())
print(len(tweets))
# tweets = collect_results(rule,max_results=500,result_stream_args=premium_search_args)

1676


In [230]:
timeline_df = pd.DataFrame(tweets)
timeline_df = timeline_df[['id','created_at', 'source', 'text','extended_tweet','favorite_count', 'retweet_count']]
timeline_df["original_author"],timeline_df["type"] = username, "tweet"
print(timeline_df.shape,list(timeline_df.columns))
timeline_df.head()

(1676, 9) ['id', 'created_at', 'source', 'text', 'extended_tweet', 'favorite_count', 'retweet_count', 'original_author', 'type']


,id,created_at,source,text,extended_tweet,favorite_count,retweet_count,original_author,type
0,1186053000843976704,Sun Oct 20 22:54:05 +0000 2019,"<a href=""http://twitter.com/download/iphone"" r...",I am so proud of our candidates.\n\nYou showed...,{'full_text': 'I am so proud of our candidates...,2726,879,MaximeBernier,tweet
1,1186022380839362562,Sun Oct 20 20:52:25 +0000 2019,"<a href=""http://twitter.com/download/iphone"" r...",Dernier jour de campagne en Beauce! \nLast day...,NaN,464,133,MaximeBernier,tweet
2,1185994970517381120,Sun Oct 20 19:03:30 +0000 2019,"<a href=""http://twitter.com/download/iphone"" r...",Imagine how Canadian politics would be insipid...,{'full_text': 'Imagine how Canadian politics w...,1161,531,MaximeBernier,tweet
3,1185990944534994944,Sun Oct 20 18:47:30 +0000 2019,"<a href=""http://twitter.com/download/iphone"" r...",Are you tired of the constant pandering?\n\nTh...,{'full_text': 'Are you tired of the constant p...,2059,771,MaximeBernier,tweet
4,1185953045588381697,Sun Oct 20 16:16:54 +0000 2019,"<a href=""http://twitter.com/download/iphone"" r...",Don’t vote out of fear.\n\nVote for your value...,NaN,1595,666,MaximeBernier,tweet


In [231]:
def clean_tweet(text,extended_tweet):
    if pd.isna(extended_tweet):
        return pd.Series([clean_text(text), text])
    to_dict = dict(extended_tweet)
    return pd.Series([clean_text(to_dict["full_text"]),to_dict["full_text"]])

In [232]:
timeline_df[["text","extended_tweet"]] = timeline_df.apply(lambda x: clean_tweet(x["text"],x["extended_tweet"]),axis=1)
timeline_df["clean_text"] = timeline_df["text"]
timeline_df["original_text"] = timeline_df["extended_tweet"]
timeline_df = timeline_df.drop(["text","extended_tweet"],axis=1)
timeline_df.head()

,id,created_at,source,favorite_count,retweet_count,original_author,type,clean_text,original_text
0,1186053000843976704,Sun Oct 20 22:54:05 +0000 2019,"<a href=""http://twitter.com/download/iphone"" r...",2726,879,MaximeBernier,tweet,proud candidate showed incredible courage pass...,I am so proud of our candidates.\n\nYou showed...
1,1186022380839362562,Sun Oct 20 20:52:25 +0000 2019,"<a href=""http://twitter.com/download/iphone"" r...",464,133,MaximeBernier,tweet,dernier jour campagne beauce last day campaign...,Dernier jour de campagne en Beauce! \nLast day...
2,1185994970517381120,Sun Oct 20 19:03:30 +0000 2019,"<a href=""http://twitter.com/download/iphone"" r...",1161,531,MaximeBernier,tweet,imagine politics would insipid irrelevant with...,Imagine how Canadian politics would be insipid...
3,1185990944534994944,Sun Oct 20 18:47:30 +0000 2019,"<a href=""http://twitter.com/download/iphone"" r...",2059,771,MaximeBernier,tweet,tired constant pandering party campaigned good...,Are you tired of the constant pandering?\n\nTh...
4,1185953045588381697,Sun Oct 20 16:16:54 +0000 2019,"<a href=""http://twitter.com/download/iphone"" r...",1595,666,MaximeBernier,tweet,vote fear vote value vote believe,Don’t vote out of fear.\n\nVote for your value...


In [233]:
csvFile = open(file_path, 'w' ,encoding='utf-8')
timeline_df.to_csv(csvFile, mode='w', index=False, encoding="utf-8")